In [1]:
import pandas as pd
from tqdm import tqdm
import os
import pickle
import cv2
from hand_tracker import HandTracker
import numpy as np

In [2]:
# 基础配置
root_dir = 'D:\\jester\\'
data_dir = 'D:\\jester\\data\\'
label_fn = 'jester-v1-labels.csv'
train_fn = 'jester-v1-train.csv'
test_fn = 'jester-v1-test.csv'
valid_fn = 'jester-v1-validation.csv'

train_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\train.pkl'
valid_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\valid.pkl'

train_size = 5000
valid_size = 200

joint_num = 21
joint_dim = 2

In [3]:
# 模型相关初始化
palm_model_path = ".\\models\\palm_detection.tflite"
landmark_model_path = ".\\models\\hand_landmark.tflite"
anchors_path = ".\\data\\anchors.csv" 
# box_shift determines 
from hand_tracker import HandTracker
detector = HandTracker(palm_model_path, landmark_model_path, anchors_path,
                       box_shift=0.2, box_enlarge=1.3)

hand tracker built ok


In [4]:
# 加载动作类别
cls_csv = pd.read_csv(root_dir + label_fn, sep=';')
cls_dict = dict(zip(cls_csv.clsid, cls_csv.clsname))
cls_dict_verse = dict(zip(cls_csv.clsname, cls_csv.clsid))
# print(cls_dict[1], cls_dict_verse['Swiping Left'])

In [5]:
print(cls_dict)

{1: 'Swiping Left', 2: 'Swiping Right', 3: 'Swiping Down', 4: 'Swiping Up', 5: 'Pushing Hand Away', 6: 'Pulling Hand In', 7: 'Sliding Two Fingers Left', 8: 'Sliding Two Fingers Right', 9: 'Sliding Two Fingers Down', 10: 'Sliding Two Fingers Up', 11: 'Pushing Two Fingers Away', 12: 'Pulling Two Fingers In', 13: 'Rolling Hand Forward', 14: 'Rolling Hand Backward', 15: 'Turning Hand Clockwise', 16: 'Turning Hand Counterclockwise', 17: 'Zooming In With Full Hand', 18: 'Zooming Out With Full Hand', 19: 'Zooming In With Two Fingers', 20: 'Zooming Out With Two Fingers', 21: 'Thumb Up', 22: 'Thumb Down', 23: 'Shaking Hand', 24: 'Stop Sign', 25: 'Drumming Fingers', 26: 'No gesture', 27: 'Doing other things'}


## 加载训练集

In [6]:
try:
    Train = pickle.load(open(train_output_fn, "rb")) # 先加载一波旧数据
except:
    Train = {}
    Train['pose'] = []
    Train['label'] = []
    
print(len(Train['label']))

680


In [7]:
# 加载训练集
train_data = pd.read_csv(root_dir + train_fn, sep=';')
train_dict = dict(zip(train_data.id, train_data.clsid))

for index in tqdm(list(train_dict.items())[len(Train['label']):train_size]):
    image_path = data_dir + str(index[0])
    image_file_list = os.listdir(image_path)    
    # 逐个加载并用 HandTracker 得出骨架数据
    skeleton_list = np.empty((len(image_file_list),(joint_dim * joint_num)), dtype = float, order = 'C')
    for file_index in range(len(image_file_list)):
        file = image_file_list[file_index]
        image = cv2.imread(image_path + '\\' + file)
        kp, box = detector(image)
        if kp is None:        
            # 暂时按0处理
            kp = np.zeros(joint_dim * joint_num)
        else:
            kp = kp.reshape(joint_dim * joint_num)
        skeleton_list[file_index] = kp
    Train['pose'].append(skeleton_list)
    Train['label'].append(index[1])



# TODO: 空骨架 --> 开头(0,0,0)，中间插值
# TODO: 中值滤波

# TODO: 统计空骨架的概率

  0%|                                                                                         | 0/4320 [00:00<?, ?it/s]C:\Users\zys98\Desktop\graduation\github-repo\gg_handtracking\python-handtracking\hand_tracker.py:96: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x) )
 13%|█████████▍                                                                | 554/4320 [2:40:46<18:12:54, 17.41s/it]

KeyboardInterrupt: 

In [8]:
# Train数据保存到pickle文件
print('已保存训练数据:', len(Train['label']))
pickle.dump(Train, open(train_output_fn, "wb"))

已保存训练数据: 1234


## 加载验证集

In [10]:
try:
    Valid = pickle.load(open(valid_output_fn, "rb")) # 先加载一波旧数据
except:
    Valid = {}
    Valid['pose'] = []
    Valid['label'] = []
    
print(len(Valid['label']))

139


In [57]:
# 加载验证集
valid_data = pd.read_csv(root_dir + valid_fn, sep=';')

for index in tqdm(valid_data[len(Valid['label']):valid_size].itertuples()):
    image_path = data_dir + str(index.id)
    image_file_list = os.listdir(image_path)    
    # 逐个加载并用 HandTracker 得出骨架数据
    skeleton_list = np.empty((len(image_file_list),(joint_dim * joint_num)), dtype = float, order = 'C')
    for file_index in range(len(image_file_list)):
        file = image_file_list[file_index]
        image = cv2.imread(image_path + '\\' + file)
        kp, box = detector(image)
        if kp is None:        
            # 暂时按0处理
            kp = np.zeros(joint_dim * joint_num)
        else:
            kp = kp.reshape(joint_dim * joint_num)
        skeleton_list[file_index] = kp
    Valid['pose'].append(skeleton_list)
    Valid['label'].append(cls_dict_verse[index.clsname])



# TODO: 空骨架 --> 开头(0,0,0)，中间插值
# TODO: 中值滤波





0it [00:00, ?it/s]



1it [00:20, 20.43s/it]



2it [00:37, 18.99s/it]



3it [00:51, 17.26s/it]



4it [01:05, 16.41s/it]



5it [01:21, 16.35s/it]



6it [01:38, 16.46s/it]



7it [01:56, 16.68s/it]



8it [02:15, 16.90s/it]



9it [02:34, 17.12s/it]



10it [02:51, 17.11s/it]



11it [03:05, 16.90s/it]



12it [03:22, 16.88s/it]



13it [03:39, 16.86s/it]



14it [03:53, 16.66s/it]



15it [04:12, 16.81s/it]



16it [04:30, 16.93s/it]



17it [04:49, 17.04s/it]



18it [05:08, 17.14s/it]



19it [05:23, 17.03s/it]



20it [05:40, 17.03s/it]



21it [05:56, 16.99s/it]



22it [06:12, 16.91s/it]



23it [06:30, 16.99s/it]



24it [06:43, 16.83s/it]



25it [07:03, 16.95s/it]



26it [07:23, 17.04s/it]



27it [07:38, 16.99s/it]



28it [07:51, 16.85s/it]



29it [08:10, 16.92s/it]



30it [08:24, 16.82s/it]



31it [08:38, 16.74s/it]



32it [08:56, 16.75s/it]



33it [09:12, 16.73s/it]



34it [09:30, 16.77s/it]



35it [09:44, 16.70s/it]



36it [09:58, 16.64s/it]



37it [10:13

KeyboardInterrupt: 

In [11]:
# Valid数据保存到pickle文件
print('已保存验证数据:', len(Valid['label']))
pickle.dump(Valid, open(valid_output_fn, "wb"))

已保存验证数据: 139


## 删除空frame的数据

In [12]:

# 训练集
train_noempty_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\train_noempty.pkl'
Train_NoEmpty = pickle.load(open(train_output_fn, "rb")) # 先加载一波旧数据
poses = Train_NoEmpty['pose']

for pose_index in range(len(poses)):
    frame_index = 0
    while frame_index < len(poses[pose_index]):
        if poses[pose_index][frame_index][0] == 0:
            poses[pose_index] = np.delete(poses[pose_index], frame_index, axis=0)
        else:
            frame_index += 1
    if len(poses[pose_index]) == 0:
        poses[pose_index] = np.zeros((1,joint_num*joint_dim))
    pose = np.zeros((1,joint_num*joint_dim))
    
print('已保存(无空帧)训练数据:', len(Train_NoEmpty['label']))
pickle.dump(Train_NoEmpty, open(train_noempty_output_fn, "wb"))

已保存(无空帧)训练数据: 1234


In [13]:
# 验证集
valid_noempty_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\valid_noempty.pkl'
Valid_NoEmpty = pickle.load(open(valid_output_fn, "rb")) # 先加载一波旧数据
poses = Valid_NoEmpty['pose']

for pose_index in range(len(poses)):
    frame_index = 0
    while frame_index < len(poses[pose_index]):
        if poses[pose_index][frame_index][0] == 0:
            poses[pose_index] = np.delete(poses[pose_index], frame_index, axis=0)
        else:
            frame_index += 1
    if len(poses[pose_index]) == 0:
        poses[pose_index] = np.zeros((1,joint_num*joint_dim))
    pose = np.zeros((1,joint_num*joint_dim))
    
print('已保存(无空帧)验证数据:', len(Valid_NoEmpty['label']))
pickle.dump(Valid_NoEmpty, open(valid_noempty_output_fn, "wb"))

已保存(无空帧)验证数据: 139


## 1-6 class 删除空骨架帧 的数据

In [15]:
# 训练集
train_1to6_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\train_1to6.pkl'
Train_1to6 = pickle.load(open(train_noempty_output_fn, "rb")) # 先加载一波旧数据
poses = Train_1to6['pose']
labels = Train_1to6['label']
pose_index=0
max_cls = 6

while pose_index < len(labels):
    if labels[pose_index] <= max_cls:
        pose_index += 1
    else:
        poses.pop(pose_index)
        labels.pop(pose_index)

print('已保存(class 1-6)训练数据:', len(Train_1to6['label']))
pickle.dump(Train_1to6, open(train_1to6_output_fn, "wb"))

已保存(class 1-6)训练数据: 247


In [17]:
# 验证集
valid_1to6_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\valid_1to6.pkl'
Valid_1to6 = pickle.load(open(valid_noempty_output_fn, "rb")) # 先加载一波旧数据
poses = Valid_1to6['pose']
labels = Valid_1to6['label']
pose_index=0
max_cls = 6

while pose_index < len(labels):
    if labels[pose_index] <= max_cls:
        pose_index += 1
    else:
        poses.pop(pose_index)
        labels.pop(pose_index)

print('已保存(class 1-6)验证数据:', len(Valid_1to6['label']))
pickle.dump(Valid_1to6, open(valid_1to6_output_fn, "wb"))

已保存(class 1-6)验证数据: 33


## 用于统计各数据集空骨架的概率

In [22]:
frame_count = 0
empty_count = 0
all_emtpy = 0
for pose in Train_NoEmpty['pose']:
    this_frame = 0
    this_empty = 0
    for frame in pose:
        this_frame += 1
        frame_count += 1
        if frame[0] == 0:
            empty_count += 1
            this_empty += 1
    if this_empty == this_frame:
        all_emtpy += 1
print('总空帧率(%):',empty_count / frame_count * 100)
print('空样本率(%)',all_emtpy / len(Train_1to6['pose']) * 100)

总空帧率(%): 0.0611537677515798
空样本率(%) 7.28744939271255
